# Implementing tfSFH

In this notebook we discuss the implementation of *tfSFH*, a variant of the SF algorithm with a loss function given by the minimization of the entropy of the learned representations. This implementation relies on the learning framework of *tensorflow*, and as such it requires an approximate differentiable estimate of the entropy.

Before starting we import libraries:

In [8]:
import numpy as np
import sklearn.datasets as ds
import scipy.stats as stats
import tensorflow as tf
import matplotlib.pyplot as plt

import itertools
from tqdm import tqdm

## 1. Loss function

The SF algorithm [1] defines as a loss function a sparsity metric given by the $\ell_1$-norm:
$$ \mathcal{L}(Z) = \ell_1 (Z). $$
The minimization of this quantity acts as a proxy for the minimization of the entropy of the learned representation.

tfSFH proposes to substitute this proxy with the direct minimization of the entropy:
$$ \mathcal{L}(Z) = H (Z) $$.

## 2. Entropy estimation

### Entropy estimation in information bottleneck research

Previous work on the dynamics of information-theoretic quantities in deep networks [2,3] and in sparse filtering [4] relied on a simple *binning* techniques for the estimation of information-theoretic quantities such as entropy.

More specifically, given a matrix of representations $\mathbf{Z}$ of dimension $N \times M$ ($N$ samples, $M$ dimensions), entropy is estimated according to the following steps:
1. *Binning*: each dimension is divided into $B$ bins; every feature (contained in the range $[0,1]$) is divided by $1/B$ and the result floored. In this way all the features are forced to obtain a discrete value between $[0,B]$.
2. *Projection in 1D*: all dimensions are concatenated into a single value, that producing a new representation of dimension $N \times 1$; practically this is achieved by concatenating the byte representations of the values on each dimension.  
3. *Distribution estimation*: an emprical distribution $\hat{p}(z)$ in 1D is computed by frequency counting.
4. *Entropy estimation*: entropy is evaluated using the standard formula $- \sum_z \hat{p}(z) \log \hat{p}(z)$.

This simple approach was computationally feasible (given a limited number of dimensions as in [2,4]) and robust to a range of values of bins $B$ [2].

### Entropy estimation for tfSFH

The same succesful approach does not easily lend itself to tfSFH. The discrete approach we described (notice the flooring operation in step 1 and the counting operation in step 2) do not easily fit in the automatic differentiation framework of tensorflow; indeed gradient would not flow through such operations.

There are non-trivial challenges that we need to consider:
- We can not use discrete non-differentiable functions (like *floor()*) for binning; instead we need to approximate binning with a continuous function.
- We can not trivially project out $N$-dimensional data to 1D; since we can not obtain discrete values, but we need to work with real values, it is hard to devise a differentiable bijective mapping from $\mathbb{R}^N$ to $\mathbb{R}$ (is this possible at all?)

What we aim at doing is getting **approximate discrete entropy estimation using a continuous function**. For this reason we will rely on *kernel density estimation* (KDE). However, we will not use KDE to estimate a continuous pdf, as this will present the further problem of integrating the differential entropy. We will apply a kernel on a discrete set of points on a $N$-dimensional grid, estimate an empirical distribution in 1D and compute its entropy.

More precisely we define the following algorithm:
1. *Gridding*: we generate a grid in $N$ dimension with resolution $B$, that is, in every dimension, the axis $[0,1]$ is discritezed in $B$ points.
2. *KDE*: we superimpose a Gaussian kernel with bandwidth $w$ over each point of the grid
3. *Distribution estimation*: we sum up and normalize the result of KDE to get an an emprical distribution $\hat{p}(z)$ in 1D.
4. *Entropy estimation*: entropy is evaluated using the standard formula $- \sum_z \hat{p}(z) \log \hat{p}(z)$.

This approach is clearly subject to the **curse of dimensionality** and wouldn't scale to high-dimensional problems. However, it provides a tool to probe the dynamics of SF. Moreover, because of the specific form of the SF algorithms, we can take advantage of a **simple manifold trick** which allows us to estimate entropy in a $(N-1)$-dimensional space; this is possible because SF projects all the data on a (N-1) dimensional manifold.

## 3. Comparison of entropy estimation via binning and gridding

We compare the two estimation techniques on a set of test distributions.

First we define the two functions to estimate entropy:

In [2]:
def binning_entropy(x,bins=30):
    binsize = 1. / bins
    digitized = np.floor(x / binsize).astype('int')
    
    uniqueids = np.ascontiguousarray(digitized).view(np.dtype((np.void, digitized.dtype.itemsize * digitized.shape[1])))
    
    _, _, unique_counts = np.unique(uniqueids, return_index=False, return_inverse=True, return_counts=True)
    px = np.asarray(unique_counts / float(sum(unique_counts)))
    
    return -np.sum(px * np.log(px))

In [3]:
def gridding_entropy(x,bins=30,sigma2=1./32):
    axis = np.linspace(0,1,30)
    grid = itertools.product(axis,repeat=x.shape[1])
        
    c1 = 1 / np.sqrt(2*np.pi*sigma2)
    c2 = 2*sigma2
    kde = np.array(list( map( lambda t: np.sum(  c1 * np.exp(-(x-t)**2 / c2 )), grid) ))
    
    px = kde / np.sum(kde)
    
    return -np.sum(px * np.log(px))
    

We then define a helper method to run our tests:

In [64]:
def compare(datagenerator,n_simuls=20,n_samples=1000,bin_bins=30, grid_bins=30, grid_sigma2=1./32):
    
    bin_estimates = []; grid_estimates = []
    for _ in tqdm(range(n_simuls)):
        X = datagenerator(n_samples)
        bin_estimates.append(binning_entropy(X,bins=bin_bins))
        grid_estimates.append(gridding_entropy(X,bins=grid_bins,sigma2=grid_sigma2))
        
    print('Avg+-std binH: {0} +- {1}'.format(np.mean(bin_estimates),np.std(bin_estimates)))
    print('Avg+-std grdH: {0} +- {1}'.format(np.mean(grid_estimates),np.std(grid_estimates)))
        
    return bin_estimates,grid_estimates

Next we run a series of tests using different types of distributions. Notice that our method assumes that all the data points are in the range $[0,1]$ (this assumptions is due to the property of SF, where we will apply our algorithm); so we will select distributions generating point with high likelihood in that interval, although they will not be strictly guaranteed not to generate points beyond that range.

### 2D Normal (.5,.2)

In [30]:
mu = np.array([.5,.5])
sigma = np.array([[.2,0],[0,.2]])
model = stats.multivariate_normal(mean=mu, cov=sigma)

binH, gridH = compare(model.rvs)    

100%|██████████| 20/20 [00:00<00:00, 20.45it/s]

Avg+-std binH: 6.63496924989536 +- 0.015149046367761662
Avg+-std grdH: 6.795492959472905 +- 0.0009436555507206885


In two dimensions, with a limited number of samples (1000), estimates of entropy are quite close.

### 2D Normal (.5,$\Sigma$)

In [31]:
mu = np.array([.5,.5])
sigma = ds.make_spd_matrix(2)
model = stats.multivariate_normal(mean=mu, cov=sigma)

print('Full covariance matrix: {0}'.format(sigma))

binH, gridH = compare(model.rvs)

 10%|█         | 2/20 [00:00<00:01, 14.49it/s]

Full covariance matrix: [[ 2.32236462 -0.76461504]
 [-0.76461504  0.87846379]]


100%|██████████| 20/20 [00:00<00:00, 20.16it/s]

Avg+-std binH: 6.860609387993874 +- 0.008095547787326927
Avg+-std grdH: 6.801550227133106 +- 0.0005743369866285584


Similar results are obtained using a Normal distribution with a full covariance matrix.

### 2D Uniform (0,1)

In [32]:
def model(n_samples):
    model1 = stats.uniform(loc=0.,scale=1.)
    model2 = stats.uniform(loc=0.,scale=1.)
    x1 = np.expand_dims(model1.rvs(1000),axis=1)
    x2 = np.expand_dims(model2.rvs(1000),axis=1)
    return np.hstack((x1,x2))

binH, gridH = compare(model)    

100%|██████████| 20/20 [00:00<00:00, 20.15it/s]

Avg+-std binH: 6.287658777161563 +- 0.013158338559071883
Avg+-std grdH: 6.793125125744642 +- 0.0010229753340355037


On uniform distribution we notice a slightly larger gap, with the binning method actually returning a high standard deviation.

### 2D Delta(0.5) 

In [34]:
def model(n_samples):
    return np.ones((n_samples,2))/2.

binH, gridH = compare(model)  

100%|██████████| 20/20 [00:00<00:00, 21.11it/s]

Avg+-std binH: 0.0 +- 0.0
Avg+-std grdH: 6.61961586586378 +- 0.0


The delta function highlights the main problem of the gridding technique. Although the samples always have the same value (and the binning method here correctly returns zero entropy), the kernel practically spreads the probability of each point over its bandwidth. As such, entropy is not zero. This would be very problematic if we were aiming for an unbiased estimate of the entropy. However, we still observe that this entropy is lower than any other entropy we have computed so far; in the context of learning and minimizing entropy, the bias may then be less important.

### [1D Laplace(.6,.2), 1D Laplace(.3,.1)]

In [35]:
def model(n_samples):
    model1 = stats.laplace(loc=.6,scale=.2)
    model2 = stats.laplace(loc=.3,scale=.1)
    x1 = np.expand_dims(model1.rvs(1000),axis=1)
    x2 = np.expand_dims(model2.rvs(1000),axis=1)
    return np.hstack((x1,x2))

binH, gridH = compare(model)    

100%|██████████| 20/20 [00:00<00:00, 20.23it/s]

Avg+-std binH: 5.744703229010865 +- 0.026088726678866563
Avg+-std grdH: 6.700316384760887 +- 0.002420943002890105


Two concentrate Laplace distributions have very low entropy. We observe again the same dynamics as before: the binning estimate is low, and the gridding estimate approaches the lower bound ($6.61$) set by the kernel bandwidth and computed in the previous simulation.

### 3D Normal (.5,.2)

In [36]:
mu = np.array([.5,.5,.5])
sigma = np.array([[.2,0,0],[0,.2,0],[0,0,.2]])
model = stats.multivariate_normal(mean=mu, cov=sigma)

binH, gridH = compare(model.rvs)    

100%|██████████| 20/20 [00:36<00:00,  1.84s/it]

Avg+-std binH: 6.900754492458478 +- 0.0026110588402981014
Avg+-std grdH: 10.199041586005245 +- 0.0005408577214619009


In three dimensions, with the same number of samples (1000), the curse of dimensionality already affects the estimate.

In [38]:
mu = np.array([.5,.5,.5])
sigma = np.array([[.2,0,0],[0,.2,0],[0,0,.2]])
model = stats.multivariate_normal(mean=mu, cov=sigma)

binH, gridH = compare(model.rvs,n_samples=10000)    

100%|██████████| 20/20 [05:47<00:00, 17.39s/it]

Avg+-std binH: 9.147613595352007 +- 0.0031304220015468094
Avg+-std grdH: 10.199059394093913 +- 0.00013264980742082685


A higher number of samples ($10^4$) reduces the gap between the estimates. Notice though, that it is the *binning estimate* that changes the most, suggesting that the *gridding* estimate was quite reliable already with less samples.

### 2D Normal (.5,.2) in low-data regime

In [65]:
mu = np.array([.5,.5])
sigma = np.array([[.2,0],[0,.2]])
model = stats.multivariate_normal(mean=mu, cov=sigma)

binH, gridH = compare(model.rvs,n_simuls=100,n_samples=5)    

100%|██████████| 100/100 [00:01<00:00, 83.65it/s]

Avg+-std binH: 1.6066653237118595 +- 0.027586909468920784
Avg+-std grdH: 6.739610541224207 +- 0.040330259449347344


## 4. Implementing the gridding method in tensorflow

We implement a new version of *tfSF* named **tfSFH**. The code is identical to the *tfSF* module, except for a new custom loss, **Hloss()**. *Hloss()* is a parametric loss, which receives the grid over which to apply the kde, and the bandwidth of the kernel.

The implementation of the loss closely follow the *gridding_entropy()* function defined above, with two minor exceptions: (i) it calls tensorflow/keras methods instead of numpy methods; (ii) it discount one dimension, since the representation learned by SF is bound on a manifold.

## Conclusions

- Gridding method seems more biased than binning, but slightly less affected by the curse of dimensionality.
- Gridding method is implemented in TF

### References
[1] Ngiam, J., Chen, Z., Bhaskar, S.A., Koh, P.W. and Ng, A.Y., 2011. Sparse filtering. In Advances in neural information processing systems (pp. 1125-1133).

[2] Shwartz-Ziv, R. and Tishby, N., 2017. Opening the black box of deep neural networks via information. arXiv preprint arXiv:1703.00810.

[3] Saxe, A.M., Bansal, Y., Dapello, J., Advani, M., Kolchinsky, A., Tracey, B.D. and Cox, D.D., 2019. On the information bottleneck theory of deep learning. Journal of Statistical Mechanics: Theory and Experiment, 2019(12), p.124020.

[1] Zennaro, F.M. and Chen, K., 2019. Towards Further Understanding of Sparse Filtering via Information Bottleneck. arXiv preprint arXiv:1910.08964.